# LAQN Updated Acvtive sites/species File Function:
- laqn_remove notebook got slower so i will move the update function here.
- Start with paths and modules to import.

In [1]:
import pandas as pd
from pathlib import Path
import os
import re

# paths beloww
base_dir = Path("/Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels")
log_path = base_dir / "data" / "laqn" / "missing" / "logs_missin_value.csv"



# function to remove non-active sites/species from active metadata csv file I used in get_laqn fetching script
active_path = base_dir / "data" / "laqn" / "actv_sites_species.csv"
nonactive_path = base_dir / "data" / "laqn" / "missing" / "nonActive_siteSpecies.csv"
output_path = base_dir / "data" / "laqn" / "updated_actv_siteSpecies.csv"

### 1) function to remove non-active site/species from active metadata csv file.
- what does function:
    - removes rows from actv_sites_species.csv where (SiteName, SpeciesCode) matches any (siteCode, SpeciesCode) in nonActive_siteSpecies.csv
    - result is saved as updated_actv_siteSpecies.csv in the same directory

- paths:
    - actv_sites_species.csv: base_dir / data / laqn / actv_sites_species.csv
    - nonActive_siteSpecies.csv: base_dir / data / laqn / missing / nonActive_siteSpecies.csv
    - updated_actv_siteSpecies.csv: base_dir / data / laqn / updated_actv_siteSpecies.csv

- new csv file save as:updated_actv_siteSpecies.csv



In [2]:
def remove_nonactive_from_active(active_path: Path, nonactive_path: Path, output_path: Path):
    """
    Remove non-active site/species combinations from active dataset.
    Args:
        active_path: Path to active site/species CSV
        nonactive_path: Path to non-active site/species CSV
        output_path: Where to save filtered active CSV
    """
    active_df = pd.read_csv(active_path)
    nonactive_df = pd.read_csv(nonactive_path)

    # Create a set of (siteCode, SpeciesCode) tuples for non-active
    nonactive_set = set(zip(nonactive_df['siteCode'], nonactive_df['SpeciesCode']))

    # Filter active_df to exclude non-active combinations
    filtered_active = active_df[
        ~active_df.apply(lambda row: (row['SiteCode'], row['SpeciesCode']) in nonactive_set, axis=1)
    ].copy()

    # Calculate what was removed
    removed_count = len(active_df) - len(filtered_active)
    
    # Save the cleaned dataset
    filtered_active.to_csv(output_path, index=False)


    # Save the filtered DataFrame
    filtered_active.to_csv(output_path, index=False)
    print(f"Filtered active data saved to {output_path}. Shape: {filtered_active.shape}")

    return filtered_active

In [3]:
# Run the function
cleaned_df = remove_nonactive_from_active(
    active_path=active_path,
    nonactive_path=nonactive_path,
    output_path=output_path
)

# Optional: Check what got removed
print("\n" + "="*60)
print("Chercking removed site/species combinations:")
print("="*60)

# Load original active list
original = pd.read_csv(active_path)

# Find what was removed (for verification)
removed_combinations = original[
    ~original.apply(
        lambda row: (row['SiteCode'], row['SpeciesCode']) in set(
            zip(cleaned_df['SiteCode'], cleaned_df['SpeciesCode'])
        ),
        axis=1
    )
][['SiteCode', 'SpeciesCode', 'SiteName']].drop_duplicates()

print(f"\nRemoved {len(removed_combinations)} unique site-species combinations:")
print(removed_combinations.to_string(index=False))

Filtered active data saved to /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/updated_actv_siteSpecies.csv. Shape: (216, 7)

Chercking removed site/species combinations:

Removed 36 unique site-species combinations:
SiteCode SpeciesCode                                 SiteName
     BX1          CO                     Bexley - Slade Green
     BX1        PM10                     Bexley - Slade Green
     BT4         SO2                             Brent - Ikea
     BT6         SO2        Brent - John Keble Primary School
     BY7          CO                 Bromley - Harwood Avenue
     BL0          CO                      Camden - Bloomsbury
     CR9         NO2                      Croydon - Park Lane
     CR9        PM10                      Croydon - Park Lane
     EN4        PM10                     Enfield - Derby Road
     EN4         SO2                     Enfield - Derby Road
     GR4         SO2                       Greenwich - Eltham
   